In [ ]:
code_departement = "971"
logs_level = "WARNING"

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import Point

from potentiel_solaire.constants import DEFAULT_CRS, CRS_FOR_BUFFERS
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, get_topo_buildings_of_interest
from potentiel_solaire.sources.schools_establishments import extract_schools_establishments, get_schools_establishments_of_interest
from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.features.pvgis_api import get_potentiel_solaire_from_pvgis_api
from potentiel_solaire.features.solar_potential import calculate_surface_utile, calculate_installed_capacity
from potentiel_solaire.logger import get_logger

logger = get_logger()
logger.setLevel(logs_level)

In [ ]:
schools_establishments_path = extract_schools_establishments()
print(f"Annuaire des établissements scolaires extrait ici: {schools_establishments_path}")

In [ ]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

In [ ]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=schools_establishments_path,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

In [ ]:
codes_commune = schools_establishments["code_commune"].unique()
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
communes = communes[communes.code_insee.isin(codes_commune)]
geom_of_interest = communes.dissolve()[["geometry"]]

In [ ]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

In [ ]:
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

In [ ]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
).sample(100)
nb_schools_buildings = schools_buildings.shape[0]
print("Nb de batiments scolaires: ", nb_schools_buildings)

In [ ]:
# calcul du potentiel en prenant les positions les plus exactes possibles
schools_buildings["longitude"] = schools_buildings.centroid.map(lambda p: p.x)
schools_buildings["latitude"] = schools_buildings.centroid.map(lambda p: p.y)

crs_init = schools_buildings.crs
schools_buildings = schools_buildings.to_crs(epsg=CRS_FOR_BUFFERS)
schools_buildings["surface_totale_au_sol"] = schools_buildings.area
schools_buildings = schools_buildings.to_crs(crs_init)

schools_buildings["surface_utile"] = schools_buildings.apply(
    lambda building: calculate_surface_utile(
        surface_totale_au_sol=building["surface_totale_au_sol"]
    ), axis=1
)

schools_buildings["peakpower"] = schools_buildings.apply(
        lambda building: calculate_installed_capacity(
            rooftop_surface=building["surface_utile"]
        ), axis=1
)

schools_buildings["potentiel_solaire_exact"] = schools_buildings.apply(
        lambda building: get_potentiel_solaire_from_pvgis_api(
            longitude=building["longitude"],
            latitude=building["latitude"],
            peakpower=building["peakpower"]
        ), axis=1
)

In [ ]:
# calcul du potentiel en prenant le centre du departement comme position
departement_center = Point(geom_of_interest.centroid.x, geom_of_interest.centroid.y)
schools_buildings["distance_to_center"] = schools_buildings.geometry.distance(departement_center)
closest_building = schools_buildings.loc[schools_buildings["distance_to_center"].idxmin()].geometry

pot = get_potentiel_solaire_from_pvgis_api(
    longitude=closest_building.centroid.x,
    latitude=closest_building.centroid.y,
    peakpower=1
)

schools_buildings["potentiel_solaire_approx"] = pot * schools_buildings["peakpower"]

In [ ]:
potentiel_solaire_exact = schools_buildings["potentiel_solaire_exact"].sum()
potentiel_solaire_approx = schools_buildings["potentiel_solaire_approx"].sum()
print(f"Utiliser la position exacte & peakpower du bâtiment donne: {potentiel_solaire_exact:.0f} kWh/an")
print(f"Utiliser le centre du departement & peakpower normalisé à 1 donne: {potentiel_solaire_approx:.0f} kWh/an")
print(f"Soit un écart de {(potentiel_solaire_approx-potentiel_solaire_exact)/(potentiel_solaire_exact):.4f}%")

In [ ]:
peakpowers = np.linspace(0, 500, 51)
potentials = [
    get_potentiel_solaire_from_pvgis_api(
        longitude=closest_building.centroid.x,
        latitude=closest_building.centroid.y,
        peakpower=peakpower
    ) for peakpower in peakpowers
]

plt.scatter(peakpowers, potentials)